### Goal of this pipeline:
Goal of this pipeline is simple! So far, we have trained two models to detect:
1. Players, Referees, Balls, GoalKeepers
2. Number in the jersey

Now, the idea is to merge these two model to get a video where the players are being tracked and provided with jersey number as their id.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
from pathlib import Path
from PIL import Image
from typing import Dict, List, Tuple
from torchvision import transforms, models
import YOLO
import numpy as np

In [ ]:
## Load the model to detect objects

detector_path = 'models/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
number_classifier_path = 'models/number_classifier.pb'

## Class to identify players by their jersey

In [ ]:
class PlayerIdentifier:
    """We have two models, one for detecting objects and one for classifying numbers. The model for classifying numbers has been trained on cropped image and hence we need to find the bounding box of the numbers in the image first and then use the number identifier on these bounding boxed image"""
    def __init__(self, object_detector_weights: str, number_classifier_weights: str, confidence_threshold: float = 0.5, device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.confidence_threshold = confidence_threshold

        # Load the object detector model    
        self.object_detector = YOLO(object_detector_weights).to(self.device)

        # Load the number classifier model based on RESNET 18 model
        self.jersey_recognizer = JerseyNumberRecognizer(number_classifier_weights).to(self.device)

        self.class_names = ['player', 'referee', 'goalkeeper', 'ball']

    def extract_jersey_region(self, frame: np.ndarray, bbox: List[float], padding: float=0.1)-> np.ndarray:
        """Extract jersey region from the player detection"""

        # Extract the bounding box
        x1, y1, x2, y2 = bbox
        h = y2 - y1
        w = x2 - x1

        # Add padding to the bounding box
        pad_x = int(padding * w)
        pad_y = int(padding * h)

        x1 = max(0, x1 - pad_x)
        x2 = min(frame.shape[1], x2 + pad_x)
        y1 = max(0, y1 - pad_y)
        y2 = min(frame.shape[0], y2 + pad_y)

        return frame[y1:y2, x1:x2] 
    
    def process_frame(self, frame: np.ndarrya, draw: bool = False) -> Tuple[List[Dict], np.ndarray]:
        """ Process a single frame to return the list of detections with jersey numbers"""
        results = self.object_detector(frame)
        detections = []
        results = results[0]
        # Draw bounding boxes
        boxes = results.boxes.xyxy.cpu().numpy()

        for box in boxes:
            x1, y1, x2, y2 = box
            class_id = int(cls)
            
            if class_id == 0: # player class
                jersey_region = self.extract_jersey_region(frame, [x1, y1, x2, y2 ])
                
                if jersey_region is not None:
                    number, number_conf = self.jersey_recognizer.predict(jersey_region)
                else:
                    number = -1
                    number_conf = 0.0

                detection = {
                    'class': self.class_names[class_id],
                    'bbox': [x1, y1, x2, y2],
                    'number': number,
                    'number_conf': number_conf,
                    'Jersey_number': int(number)
                }
            else:
                detection = {
                    'class': self.class_names[class_id],
                    'bbox': [x1, y1, x2, y2],
                    'confidence': conf
                }



SyntaxError: incomplete input (2678833978.py, line 16)